# 9 Facial Expressions You Need

# ❇️ Importing Necessary Libraries

In [1]:
import os
import cv2
import numpy as np

- **os** → Handles file and folder operations (e.g., list, create, delete).  
- **cv2** → OpenCV library used for image processing (read, display, edit images).  
- **numpy** → Used for numerical operations and arrays (images are stored as NumPy arrays).

In [3]:
def preprocess_image(image_path, target_size=(640, 640), to_gray=False):
    img = cv2.imread(image_path)
    if img is None:
        return None
    if to_gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size) # resize

    
    # Normalize for training (not saving)
    img_normalized = img / 255.0  # Scale to 0-1 for uint8 images

    # Convert back to uint8 for saving
    img_save = (img_normalized * 255).astype("uint8")

    # Save properly
    cv2.imwrite(os.path.join(output_img_path, file), cv2.cvtColor(img_save, cv2.COLOR_RGB2BGR))

### 🧩 Function: `preprocess_image`

**Purpose:**  
Preprocesses an image before training — resizing, color conversion, and normalization.

**Steps:**
1. **Read image**
   - `cv2.imread(image_path)` loads the image.
   - Returns `None` if the file doesn't exist.

2. **Color conversion**
   - If `to_gray=True`: convert to grayscale → `cv2.COLOR_BGR2GRAY`
   - Else: convert from BGR → RGB (OpenCV loads in BGR by default).

3. **Resize**
   - `cv2.resize(img, target_size)` resizes the image (default 640×640).

4. **Normalize**
   - `img / 255.0` scales pixel values from `0–255` to `0–1` for training.

5. **Convert back to uint8**
   - `(img_normalized * 255).astype("uint8")` brings values back to 8-bit format for saving.

6. **Save processed image**
   - `cv2.imwrite()` saves the image after converting RGB → BGR again for OpenCV.

**Note:**  
- `output_img_path` and `file` should be defined before calling `cv2.imwrite()`.


In [5]:
# folder
folders = ["train", "test", "valid"]
base_path = "/Users/natashababu/Documents/internship/facial-expression/9-Facial-Expressions-You-Need"
save_path = "/Users/natashababu/Documents/internship/facial-expression/preprocessed-dataset"

for folder in folders:
    input_img_path = os.path.join(base_path, folder, "images")
    input_lbl_path = os.path.join(base_path, folder, "labels")

    output_img_path = os.path.join(save_path, folder, "images")
    output_lbl_path = os.path.join(save_path, folder, "labels")

    os.makedirs(output_img_path, exist_ok=True)
    os.makedirs(output_lbl_path, exist_ok=True)

    print(f"Processing {folder} images...")
    for file in os.listdir(input_img_path):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
            
        img_path = os.path.join(input_img_path, file)
        processed = preprocess_image(img_path)

        # Copy corresponding label file
        label_name = os.path.splitext(file)[0] + ".txt"
        src_label = os.path.join(input_lbl_path, label_name)
        dst_label = os.path.join(output_lbl_path, label_name)

        if os.path.exists(src_label):
            with open(src_label, "r") as f:
                label_data = f.read()
            with open(dst_label, "w") as f:
                f.write(label_data)

print("✅ All images preprocessed and labels copied successfully!")

Processing train images...
Processing test images...
Processing valid images...
✅ All images preprocessed and labels copied successfully!


### 🧩 Code Summary

**Purpose:**  
This code preprocesses all images and copies their corresponding label files for the `train`, `test`, and `valid` datasets, creating a ready-to-use dataset for training machine learning models.

**Steps:**
1. **Define paths**  
   - `base_path` points to the original dataset.  
   - `save_path` is where the preprocessed dataset will be stored.  
   - The code works with separate folders for `train`, `test`, and `valid`.

2. **Loop through folders**  
   - For each folder, input and output paths for images and labels are set.  
   - Output directories are created if they don’t exist.

3. **Process images**  
   - Iterate through all files in the input image folder.  
   - Skip files that are not images (`.jpg`, `.jpeg`, `.png`).  
   - Each image is preprocessed using the `preprocess_image()` function (resizing, color conversion, and normalization).

4. **Copy labels**  
   - For each image, find the corresponding `.txt` label file.  
   - Read the label content and write it to the output label folder, preserving the original file structure.

5. **Completion message**  
   - After all images and labels are processed, a success message is printed.

**Result:**  
A clean, preprocessed dataset with images resized and normalized, and all labels copied correctly, ready for training models while keeping the original folder structure intact.


In [16]:
# For PyTorch >=2.0 (with MPS support)
!pip install torch torchvision torchaudio

### 🧩 Code Explanation

**Purpose:**  
Installs PyTorch and related libraries for deep learning, specifically compatible with **PyTorch ≥2.0** and **MPS (Metal Performance Shaders) support** on macOS.

**Libraries:**

- torch → Core PyTorch library for tensor operations and building neural networks.
- torchvision → Utilities for computer vision tasks (image transformations, pretrained models, datasets).
- torchaudio → Utilities for audio processing with PyTorch (datasets, transforms, models).

In [18]:
import torch
torch.backends.mps.is_available()  # Should return True
torch.backends.mps.is_built()      # Should return True

True

### 🧩 Code Explanation

**Purpose:**  
Checks if **MPS (Metal Performance Shaders)** is available and built on your Mac for PyTorch GPU acceleration.

**Code:**
```python
import torch

# Check if MPS (Apple GPU support) is available
torch.backends.mps.is_available()  # Returns True if MPS can be used

# Check if MPS is built in your PyTorch installation
torch.backends.mps.is_built()      # Returns True if PyTorch was compiled with MPS support

In [ ]:
from ultralytics import YOLO

## 1️⃣ Load pretrained YOLOv8 nano model
# 'n' = nano → fastest and lightest for small datasets
model = YOLO("yolov8n.pt")

# 2️⃣ Training parameters optimized
model.train(
    data="data.yaml",     # Path to your dataset YAML
    epochs=10,           # Max epochs; early stopping will stop sooner
    imgsz=320,            # Image size (resize all images)
    device="mps",         # Use Apple GPU (MPS)
    patience=10,          # Early stopping if val mAP doesn't improve for 10 epochs
    batch=16,             # Adjust batch size if you get memory errors
    augment=True,         # Enable YOLO's built-in augmentation
    hsv_h=0.015,          # Hue augmentation
    hsv_s=0.7,            # Saturation
    hsv_v=0.4,            # Brightness/value
    degrees=10,           # Random rotation
    translate=0.1,        # Random translation
    scale=0.5,            # Random scaling
    shear=2.0,            # Random shear
    fliplr=0.5,           # Horizontal flip
    flipud=0.0,           # Vertical flip
    mosaic=1.0,           # Mosaic augmentation
    mixup=0.1             # Mixup augmentation
)

Ultralytics 8.3.205 🚀 Python-3.10.18 torch-2.8.0 MPS (Apple M3 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=10, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, 

### 🧩 YOLOv8 Training Code Explanation

**Purpose:**  
Load a pretrained YOLOv8 nano model and train it on a custom dataset with data augmentation and optimized parameters.

---

#### 1. Import YOLO: Loads the YOLO class from the Ultralytics library.
#### 2. Load pretrained model
- "yolov8n.pt" → nano model (smallest and fastest, ideal for small datasets).
- Pretrained weights help the model converge faster.
#### 3. Train the model
| Parameter           | Description                                              |
| ------------------- | -------------------------------------------------------- |
| `data`              | Path to dataset YAML (images, labels, class names)       |
| `epochs`            | Maximum training cycles                                  |
| `imgsz`             | Resize all images to this size                           |
| `device`            | `"mps"` → Use Apple GPU for faster training              |
| `patience`          | Early stopping if validation performance doesn’t improve |
| `batch`             | Number of images per batch                               |
| `augment`           | Enable YOLO’s built-in data augmentation                 |
| `hsv_h/s/v`         | Adjust hue, saturation, brightness randomly              |
| `degrees`           | Random rotation                                          |
| `translate`         | Random image shift                                       |
| `scale`             | Random scaling                                           |
| `shear`             | Random shear transformation                              |
| `fliplr` / `flipud` | Horizontal/vertical flips                                |
| `mosaic`            | Combine 4 images into 1 for better generalization        |
| `mixup`             | Blend two images and labels to reduce overfitting        |
